# AMPERIUM NNM (Trainer)
This notebook allows you to train a neural amp model based on a pair of input/output WAV files that you have of the amp you want to model.

Amp + Cab and only Amp supported.

Use 44100 mono input signal.

As good input signal for training we can recomend https://github.com/GuitarML/Releases/releases/download/v1.0.0/Proteus_Capture_Utility.zip

**To use this notebook**:
Go to [colab.research.google.com](https://colab.research.google.com/), select the "GitHub" tab, and select this notebook.

🔶**Before you run**🔶

Make sure to get a GPU! (Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

⚠**Warning**⚠

Google Colab GPU instances only last for 12 hours.
Plan your training accordingly!

## Steps:
1. Install everything
2. Upload audio files
3. Settings
4. Run & Export
5. Download your files

In [ ]:
#@title Step 1: Install
#@markdown Install AmperiumNNM and the other Python packages it depends on.

!git clone https://github.com/KirillDeLutherSergeev/AmperiumNNM.git '/content/AmperiumNNM'

%cd 'AmperiumNNM/'

from AudioUtils import *
from Training import *
from ExportNNM import *
import matplotlib.pyplot as plt
import os

fnameInput = ''
fnameOutput = ''

### Step 2A: Upload audio files
Upload the input (DI) and output (reamped) files you want to use as input.wav and output.wav.

In [ ]:
#@title Step 2B: Mount Google drive
#@markdown Or copy files from your google drive to /content folder.
#@markdown Find your files in drive folder and copy paths to files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Step 2B: { display-mode: "form" }

#@markdown Input file path
fnameInput = "/content/drive/MyDrive/ML/Proteus_Capture.wav" #@param {type:"string"}
#@markdown Output file path
fnameOutput = "/content/drive/MyDrive/ML/Proteus_5153_ReactIR/Proteus_5153B_ReactIR.wav" #@param {type:"string"}


!cp $fnameInput '/content/input.wav'
!cp $fnameOutput '/content/output.wav'

In [ ]:
#@title Step 3: Settings { display-mode: "form" }

#@markdown NNM file name
name = "EVH5153B_V1" #@param {type:"string"}
#@markdown Amp description
ampName = "EVH 5150III 50 Blue channel" #@param {type:"string"}
#@markdown Cab description
cabName = "None" #@param {type:"string"}
#@markdown Model type
modelType = "Amp" #@param ["Amp + Cab", "Amp"]

#@markdown Input file start offset in seconds, set to 0 to use offset in samples
offsetSeconds = 5 #@param {type:"number"}
#@markdown start offset in samples
offsetSamples = 0 #@param {type:"number"}
#@markdown Output file delay
delaySamples = 2 #@param {type:"number"}

#Training parameters
learningRate = 0.008
epsilon = 1e-08
#lossType = "mse" #@param ["mae", "mse"]
lossType = "mse"

#Dataset parameters
trainSize = 0.8

numSteps = 4096
overlap = 256
batchSize = 64

in_file = '/content/input.wav'
out_file = '/content/output.wav'

x_all, y_all, sampleRate = load_audio_data(in_file, out_file, offsetSeconds, offsetSamples, delaySamples)

x_train, y_train, x_test, y_test = prepare_dataset(x_all, y_all, numSteps=numSteps, trainSize=trainSize, overlap=overlap, batchSize=batchSize)

if modelType != "Amp":
  model = build_model(True, True, True, loss=lossType, learningRate=learningRate, epsilon=epsilon)
  modelTypeID = get_type_id(11, sampleRate)
else:
  model = build_model(False, True, False, loss=lossType, learningRate=learningRate, epsilon=epsilon)
  modelTypeID = get_type_id(10, sampleRate)

In [ ]:
#@title Step 4: Run & Export { display-mode: "form" }

#@markdown Epochs to train
epochs = 8 #@param {type:"integer"}

train_model(x_train, y_train, x_test, y_test, model, epochs, batchSize)

mfname = name+'.nnm'

export_model_to_nnm('/content/'+mfname, 1, 0, model, modelTypeID, ampName, cabName)

if os.path.exists(fnameOutput):
    export_model_to_nnm(os.path.dirname(fnameOutput)+'/'+mfname, 1, 0, model, modelTypeID, ampName, cabName)